# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots StatsBase Optim DelimitedFiles"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.5.0 on the current Colab Runtime...
2020-11-22 14:29:00 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz [105098627/105098627] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed SoftGlobalScope ─ v1.1.0
  Installed ZMQ ───────────── v1.2.1
  Installed Conda ─────────── v1.5.0
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed VersionParsing ── v1.2.0
  Installed IJulia ────────── v1.23.0
  Installed JSON ──────────── v0.21.1
  Installed JLLWrappers ───── v1.1.3
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.0.12
########################################################################

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     392.793 ms (0.00% GC)
  median time:      399.581 ms (0.00% GC)
  mean time:        418.871 ms (3.80% GC)
  maximum time:     520.873 ms (17.25% GC)
  --------------
  samples:          12
  evals/sample:     1

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CuArrays

    M_gpu = cu(M)
    @benchmark CuArrays.@sync M_gpu^2
else
    println("No GPU found.")
end

######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  7
  --------------
  minimum time:     2.510 ms (0.00% GC)
  median time:      3.976 ms (0.00% GC)
  mean time:        4.006 ms (0.04% GC)
  maximum time:     8.110 ms (24.33% GC)
  --------------
  samples:          1247
  evals/sample:     1

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

# Link Prediction with Tensor Factorization (VecHGrad method)

*Last edited: 2020-11-22*

### Theoretical tools
*Based on work from:*

Gao, Sheng, Ludovic Denoyer, and Patrick Gallinari. "Link pattern prediction with tensor decomposition in multi-relational networks." In 2011 IEEE Symposium on Computational Intelligence and Data Mining (CIDM), pp. 333-340. IEEE, 2011.

*Improvements in tensor decomposition from:*

VecHGrad: https://arxiv.org/abs/1905.12413

VecHGrad Code: https://github.com/dagrate/vechgrad

### Data

* bowling-green.american-assembly from https://github.com/pol-is/openData
  * Filled in with 0 for missing data
  * Split in two matrices, one with positive observations (1) and one with negative observations (-1).

### How to use

* Matrices with pos-neg layer data need to be uploaded to Google Colab
* Vector *a* (line 6 below) needs to be updated with dimensions of input data
  * Can also be extracted by the csv files themselves
* Run helper function cells and main program
* Resulting latent feature matrices A, B and C are the ones that minimize the weighted least squares error for the missing data.

### ToDo / Improvements

* Convert notebook to Python...
* ... or go further into Julia territory, utilizing https://github.com/Jutho/TensorOperations.jl

## Helper functions

In [18]:
### TNSRUTILS.JI FUNCTIONS

function buildCP3(A, B, C)
    I = size(A)[1] ;
    J = size(B)[1] ;
    K = size(C)[1] ;
    Xh = zeros(I,J,K) ;
#
# manage the case where R == 1
    if length( size( A ) ) == 1
        tmp = kron( C, kron( A, B' ) ) ;
        irow = 1 ;
        for lk = 1:K
            Xh[:,:,lk] += tmp[irow:irow+I-1, :] ;
            irow += I ;
        end
    else
        R = size(A)[2] ;
        for li = 1:I
        	for lj = 1:J
        		for lk = 1:K
        			for lr = 1:R
        				Xh[li,lj,lk] += A[li, lr] * B[lj, lr] * C[lk, lr] ;
        			end
        		end
        	end
        end
        #kron is highly inefficient for medium to large tensors
        #for lr = 1:R
        #    tmp = kron( C[:,lr], kron( A[:,lr], B[:,lr]' ) ) ;
        #    irow = 1 ;
        #    for lk = 1:K
        #        Xh[:,:,lk] += tmp[irow:irow+I-1, :] ;
        #        irow += I ;
        #    end
        #end
    end
    return Xh ;
#
# Last card of function buildCP3.
#
end

#function objfun(X, Xh)
#    Z = errorfun(X,Xh) ;
#    return sqrt( sum(Z) ) ;

# Weighted least squares error
function objfunweighted(X, Xh, W)
    Z = errorfun(X,Xh) ;
    for l=1:size(Z)[3]
      Z[:,:,l] = Z[:,:,l] .* W ;
    end
    return sqrt( sum(Z) ) ;
#
# Last card of function objfun
#
end

function krao(matC,matD)
    (I,F) = size(matC) ;
    (J,F1) = size(matD) ;
#
    kraomat = zeros(I*J, F) ;
    for f = 1:F
        kraom = matD[:,f] * matC[:,f]' ;
        kraomat[:,f] = kraom[:];
    end
    return kraomat ;
#
# Last card of function krao
#
end

function tnsrunfold(X, mode)
    dim = collect(size(X)) ;
    ncol = Int(cumprod(dim)[end]/dim[mode]) ;
    if mode == 1
        Z = reshape(permutedims(X, [1,2,3]), dim[mode], ncol) ;
    elseif mode == 2
        Z = reshape(permutedims(X, [2,1,3]), dim[mode], ncol) ;
    elseif mode == 3
        Z = reshape(permutedims(X, [3,1,2]), dim[mode], ncol) ;
    end
    return Z ;
#
# Last card of function tnsrunfold.
#
end

function cp3init(a, latfact)
    R = latfact ;
    A = rand(a[1], R) ;
    B = rand(a[2], R) ;
    C = rand(a[3], R) ;
    return A, B, C ;
#
# Last card of function cp3init.
#
end

function errorfun(X, Xh)
    return (X-Xh) .* (X-Xh) ;
#
# Last card of function errorfunct.
#
end

In [19]:
### CPUTILS.JI FUNCTIONS

function mat2vectcp3( A,B,C )
    I = size(A)[1] ;
    J = size(B)[1] ;
    K = size(C)[1] ;
    R = size(A)[2] ;
#
    x = zeros( I*R + J*R + K*R ) ;
    cnt = 1 ;
    for n = 1:3
        if n == 1
            elts = I*R ;
            x[cnt:elts] = vcat(A...) ;
        elseif n == 2
            elts = cnt + J*R - 1 ;
            x[cnt:elts] = vcat(B...) ;
        elseif n == 3
            elts = cnt + K*R - 1 ;
            x[cnt:elts] = vcat(C...) ;
        end
        cnt = elts + 1 ;
    end
    return x ;
#
#     Last card of function mat2veccp3.
#
end

In [20]:
### ALSCP.JI FUNCTION

function nncpals(X, rankR, A, B, C, Weights, maxiter=1000, epsobjfun=1.0E-3)
    filepath = "" ;

    Xsize = collect(size(X)) ;
    allmode = collect(1:length(Xsize)) ;
    if length(Xsize) > 3
        return println("Only third order parafac implemented") ;
    end
#
# we store the evolution of the calculation
    fk = zeros( maxiter + 1 ) ;
    tmstamp = zeros( maxiter + 1 ) ;
#
# random initialization
    # A = rand(Xsize[1], rankR) ;
    # B = rand(Xsize[2], rankR) ;
    # C = rand(Xsize[3], rankR) ;
    Xh = buildCP3(A,B,C) ;
    curvo = objfunweighted(X,Xh,Weights) ;
    print("   >> initialization    Obj Fun:") ;
    println(round(curvo, digits=3)) ;
#
    fk[1] = curvo ;
    tmstamp[1] = time() ;
#
# non-negative CP ALS iterative process
    i = 1 ;
    while i <= maxiter
        for mode = 1:length(Xsize)
#
# Hadamard product on all mode != n
            V = ones(rankR, rankR) ;
            for n = 1:length(Xsize)
                if n != mode
                    if n == 1
                        tmpV = A ;
                    elseif n == 2
                        tmpV = B ;
                    elseif n == 3
                        tmpV = C ;
                    end
                    V = V .* (tmpV' * tmpV) ;
                end
            end
#
# Khatri-Rao product on all mode != n
            if mode == 1
                W = krao(C, B) ;
            elseif mode == 2
                W = krao(C, A) ;
            elseif mode == 3
                W = krao(B, A) ;
            end
#
# non negative update
            num = (tnsrunfold(X, mode) * W) .+ 1.0E-9 ;
            if mode == 1
                denum = (A * (W' * W)) .+ 1.0E-9 ;
                A = A .* (num ./ denum) ;
            elseif mode == 2
                denum = (B * (W' * W)) .+ 1.0E-9 ;
                B = B .* (num ./ denum) ;
            elseif mode == 3
                denum = (C * (W' * W)) .+ 1.0E-9 ;
                C = C .* (num ./ denum) ;
            end
        end
#
# calculation evolution
        Xh = buildCP3(A,B,C) ;
        curvo = objfunweighted(X,Xh,Weights) ;
        fk[i+1] = curvo ;
        tmstamp[i+1] = time() ;
#
        if i%(maxiter/10) == 0 || i == maxiter
            print("   >> ", i / maxiter * 100, "%") ;
            print("     Obj. fun: ", round(curvo, digits=3)) ;
            println("") ;
            xk = mat2vectcp3(A, B, C) ;
            writedlm(string(filepath, string(i), "xk.txt"), xk, ",") ;
            writedlm(string(filepath, string(i),"fk.txt"), fk, ",") ;
            writedlm(string(filepath, string(i),"tmstamp.txt"), tmstamp, ",") ;
        end
#
        if curvo > epsobjfun
            i += 1 ;
        else
            i = maxiter + 1 ;
        end
    end
    print("Number of iterations: ") ;
    println(i - 1) ;
    Xh = buildCP3(A, B, C) ;
    print("Overall difference: ") ;
    println(round(objfunweighted(X, Xh, Weights), digits=3)) ;
#
# we save the final results of the calculation
    xk = mat2vectcp3(A, B, C) ;
    writedlm(string(filepath, string(i), "xk.txt"), xk, ",") ;
    writedlm(string(filepath, string(i),"fk.txt"), fk, ",") ;
    writedlm(string(filepath, string(i),"tmstamp.txt"), tmstamp, ",") ;
#
    return A, B, C ;
#
# Last card of function nncpals.
#
end

## Main program

In [ ]:
using StatsBase
using Optim
using DelimitedFiles
using Random

a = (2031, 896, 2) ;  # Participant-votes data, 2 layers (pos,neg)
maxitrtn = 1000 ;     # Nr of iterations
flpath = "/content/"

#sim = "cifar10" ;
#rsltn = 4 ;
#desc_bef = ["fk", "xk", "tmstamp"] ;
#desc_aft = ["fk_", "xk_", "tm_"] ;
#rs = ["_ros", "_par", "_ded", "_cp"] ;

# we initialize the tensor X and a
#a = (32, 32, 16) ;
#X = readdlm( string(flpath, "cifar10_.csv") ) ;
#X = reshape(X, a[1], a[2], a[3]) ;

# Assembling the participant-votes tensor
X = zeros(a[1], a[2], a[3]) ;
Weights = bitrand(a[1], a[2]) ;   # 1 for observations, 0 for missing links (to be predicted)
X[:,:,1] = readdlm( string(flpath, "participant-votes-dataonly-positive.csv"), ',', Int, '\n' ) ;
X[:,:,2] = readdlm( string(flpath, "participant-votes-dataonly-negative.csv"), ',', Int, '\n' ) ;

# resolution of CP
latfact = 2 ;
A, B, C = cp3init(a, latfact) ;
A, B, C = nncpals(X, latfact, A, B, C, Weights, maxitrtn) ;


#for n = 1:3
#    path1 = "" ;
#    nm1 = string(path1, string(maxitrtn), desc_bef[n], ".txt") ;
#    nm2 = string(path1, sim, rs[rsltn], "_als_", desc_aft[n], string(maxitrtn), ".txt") ;
#    mv(nm1, nm2) ;
#end

   >> initialization    Obj Fun:556.08
   >> 10.0%     Obj. fun: 224.933
   >> 

In [41]:
# Naive check on how many participant-votes observations are predicted both positive and negative (obv a mistake)
Xh = buildCP3(A, B, C) ;
counter = 0
for i = 1:size(Xh)[1]
  for j = 1:size(Xh)[2]
    if Xh[i,j,1] > 0.5 && Xh[i,j,2] < -0.5
      counter += 1 ;
    end
  end
end
print("Mistakes: ")
print(counter) ;
print(" out of ")
print(size(Xh)[1] * size(Xh)[2]) ;
println(" (", 100 * counter / (size(Xh)[1] * size(Xh)[2]), "%)") ;

Mistakes: 1485 out of 1819776 (0.08160345009495674%)
